In [1]:
# レイヤーの個数 ex) tops, pants, shoes -> LAYER = 3
import sys
import time
from constants.optimisation import EPSILON, LAYER, LAYER_NAME, TIME_STEP
from util.calc_closet_score import calc_closet_score
from util.init_closet import init_closet
from util.record_data import record_data
from util_class.score_estimater import ScoreEstimater

from util.select_max_incremental_item import select_max_incremental_item
import tomotopy as tp
from util.init_all_item import init_all_item
import torch
from torchvision import models
import torch.nn as nn
import random

In [2]:
# start = time.time()
original_all_items = init_all_item(LAYER, LAYER_NAME, 200)

# FashionItemの初期化

topic_model = tp.LDAModel.load('lda_model_topic_10.bin')
similarity_model = models.resnet18(pretrained=True)
num_ftrs = similarity_model.fc.in_features

similarity_model.fc = nn.Linear(num_ftrs, 738)

similarity_model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [ ]:
with open("data/com_e.txt", mode="w") as f:
    f.write("")
with open("data/sim.txt", mode="w") as f:
    f.write("")
with open("data/ver.txt", mode="w") as f:
    f.write("")
with open("data/mul.txt", mode="w") as f:
    f.write("")
print("data clear run")


In [3]:
COUNT = 10000
CLOSET_ITEM_NUM = 40
model = ScoreEstimater(topic_model, [[], [], []], similarity_model)

In [ ]:
for cnt in range(COUNT):
    all_items = [random.sample(original_all_items[i], CLOSET_ITEM_NUM) for i in range(LAYER)]
    # closet.append(random.sample(all_items[i], 4))
    select_items = init_closet(all_items, TIME_STEP)
    model.set_all_items(all_items)
    delta_obj = EPSILON + 1
    pre_obj = 0
    # ここから最適化
    while delta_obj >= EPSILON:
        for layer in range(LAYER):
            select_items[layer] = []
            for t in range(TIME_STEP):
                sigma, additional_item = select_max_incremental_item(select_items, all_items,layer, model)
                select_items[layer].append(additional_item)
        cur_obj = sum(calc_closet_score(select_items, model))
        delta_obj = cur_obj - pre_obj
        pre_obj = cur_obj

    score = calc_closet_score(select_items, model)
    record_data("data/com_new2.txt", score[0])
    record_data("data/ver_new2.txt", score[1])
    record_data("data/sim_new2.txt", score[2])
    record_data("data/mul_new2.txt", score[3])
    print("com:{:.2f} ver:{:.2f} sim:{:.2f} mul:{:.2f}".format(*score))
    cnt += 1
    if (cnt % 5 == 0):
        print(f"{cnt * 100 / COUNT}% 終了しました")

com:11.89 ver:2.88 sim:4.01 mul:2.36
com:11.91 ver:2.85 sim:2.55 mul:2.82
com:15.67 ver:2.98 sim:4.24 mul:5.06
com:13.33 ver:2.88 sim:4.26 mul:5.06
com:11.60 ver:2.85 sim:2.23 mul:2.08
0.05% 終了しました
com:16.50 ver:2.80 sim:1.55 mul:5.06
com:14.09 ver:2.85 sim:1.55 mul:5.06
com:14.52 ver:2.90 sim:4.47 mul:4.69
com:12.16 ver:2.67 sim:5.72 mul:3.10
com:12.98 ver:2.73 sim:4.03 mul:3.57
0.1% 終了しました
